In [33]:
import bs4
import re
import copy
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

OPENAI_API_KEY = 'sk-proj-gnB8KP0GbrXROIZmAYRKT3BlbkFJveplnj1LUDtIpQcpH1ya'
PINECONE_API_KEY = 'a7c4e75b-c3e3-4df0-862d-a9d22f030fe6'
#PINECONE_INDEX_NAME = 'testindex'
PINECONE_INDEX_NAME = 'professors'

## <b>Getting All Professors/Faculty General Information: </b>
# name, email, number, building location

In [34]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [35]:
embeddings = OpenAIEmbeddings()

In [36]:
def professor_link_info(url_class):
    query_part = '?page='
    pages = 58

    all_docs = []

    for i in range(0, pages):
        # Load, chunk and index the contents of the blog.
        link = "https://wit.edu/faculty-staff-directory" + query_part + str(i + 1)
        loader = WebBaseLoader(
            web_paths=(link,),
            bs_kwargs=dict(
                parse_only=bs4.SoupStrainer(
                    class_=(url_class)
                )
            ),
        )
        loaded_doc = loader.load()
        all_docs.extend(loaded_doc)
        
    return all_docs

In [37]:
professor_titles = professor_link_info("profile-teaser__info__title")
professor_digital_contact = professor_link_info("profile-teaser__info__contact-digital")
professor_physical_contact = professor_link_info("profile-teaser__info__contact-physical")

In [38]:
pt_copy = copy.deepcopy(professor_titles)
pdc_copy = copy.deepcopy(professor_digital_contact)
ppc_copy = copy.deepcopy(professor_physical_contact)

In [39]:
print(len(professor_titles))
print(len(professor_digital_contact))
print(len(professor_physical_contact))

58
58
58


In [40]:
professor_titles

[Document(page_content='\n\nJulito Anaya\n\n\n\nMaintenance A Worker, Physical Plant\n\n\n\n\nLenny Anderson\n\n\n\nAssociate Professor, Civil Engineering, School of Engineering\n\n\n\n\nEan Anderson\n\n\n\nStudent Success Advisor, Success Studio\n\n\n\n\nKayla Anderson\n\n\n\nStudent Success Advisor, Success Studio\n\n\n\n\nNedret Andre\n\n\n\nAdjunct Instructor, Architecture, School of Architecture and Design\n\n\n\n\nNicholas Andres\n\n\n\nAcademic Coordinator, School of Architecture and Design\n\n\n\n\nMohammed Anwaruddin\n\n\n\nAdjunct Instructor, Computer Science, School of Computing and Data Science\n\n\n\n\nEd Ardolino\n\n\n\nCoach, Athletics\n\n\n\n\nChristina Arecy\n\n\n\nAssistant Director-Diversity, Engagement and Belonging, Diversity & Social Justice Programs\n\n\n\n\nDaniel Arevalo\n\n\n\nCampus Police Officer, Public Safety\n\n\n\n\nAndreas Armenis\n\n\n\nDirector of Precollege Programs, Enrollment Management\n\n\n\n\nMallory Armstrong\n\n\n\nAthletic Trainer, Athletics\

In [41]:
first_index = copy.deepcopy(professor_titles[0])
first_index.page_content = ''
first_index.metadata = {}
blank_document = first_index
blank_document

Document(page_content='')

In [42]:
pt = []
new_professor_titles = []

for i in pt_copy:
    pt.extend(i.page_content.split('\n\n\n\n\n'))

for i in range(len(pt)):
    pt[i] = ' '.join(pt[i].split())
    blank_document.page_content = pt[i]
    new_professor_titles.append(copy.deepcopy(blank_document))

pt

['Julito Anaya Maintenance A Worker, Physical Plant',
 'Lenny Anderson Associate Professor, Civil Engineering, School of Engineering',
 'Ean Anderson Student Success Advisor, Success Studio',
 'Kayla Anderson Student Success Advisor, Success Studio',
 'Nedret Andre Adjunct Instructor, Architecture, School of Architecture and Design',
 'Nicholas Andres Academic Coordinator, School of Architecture and Design',
 'Mohammed Anwaruddin Adjunct Instructor, Computer Science, School of Computing and Data Science',
 'Ed Ardolino Coach, Athletics',
 'Christina Arecy Assistant Director-Diversity, Engagement and Belonging, Diversity & Social Justice Programs',
 'Daniel Arevalo Campus Police Officer, Public Safety',
 'Andreas Armenis Director of Precollege Programs, Enrollment Management',
 'Mallory Armstrong Athletic Trainer, Athletics',
 'Joe Aronov Assistant Track & Field Coach, Athletics',
 'Everett Askerneese Maintenance A Worker, Physical Plant',
 'Ben Assa Data Architect, Digital Technology S

In [43]:
new_professor_titles

[Document(page_content='Julito Anaya Maintenance A Worker, Physical Plant'),
 Document(page_content='Lenny Anderson Associate Professor, Civil Engineering, School of Engineering'),
 Document(page_content='Ean Anderson Student Success Advisor, Success Studio'),
 Document(page_content='Kayla Anderson Student Success Advisor, Success Studio'),
 Document(page_content='Nedret Andre Adjunct Instructor, Architecture, School of Architecture and Design'),
 Document(page_content='Nicholas Andres Academic Coordinator, School of Architecture and Design'),
 Document(page_content='Mohammed Anwaruddin Adjunct Instructor, Computer Science, School of Computing and Data Science'),
 Document(page_content='Ed Ardolino Coach, Athletics'),
 Document(page_content='Christina Arecy Assistant Director-Diversity, Engagement and Belonging, Diversity & Social Justice Programs'),
 Document(page_content='Daniel Arevalo Campus Police Officer, Public Safety'),
 Document(page_content='Andreas Armenis Director of Precol

In [44]:
pdc_copy

[Document(page_content='\n\n\n\n\n      anayaj@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-4182\n    \n\n\n\n\n\n\n      andersonl@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-5008\n    \n\n\n\n\n\n\n      andersone10@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-4296\n    \n\n\n\n\n\n\n      andersonk12@wit.edu\n    \n\n\n\n\n\n\n\n      andren@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-4457\n    \n\n\n\n\n\n\n      andresn@wit.edu\n    \n\n\n\n\n\n\n\n      anwaruddinm@wit.edu\n    \n\n\n\n\n\n\n\n      ardolinoe@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-4808\n    \n\n\n\n\n\n\n      arecyc@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-4537\n    \n\n\n\n\n\n\n      arevalod@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-4003\n    \n\n\n\n\n\n\n      armenisa@wit.edu\n    \n\n\n\n\n\n\n\n      617-989-5341\n    \n\n\n\n\n\n\n      armstrongm2@wit.edu\n    \n\n\n', metadata={'source': 'https://wit.edu/faculty-staff-directory?page=1'}),
 Document(page_content='\n\n\n\n\n      aronovj@wit.edu\n    \

In [45]:
pdc = []
new_professor_digital = []
for i in range(len(pdc_copy)):
    data = ' '.join(pdc_copy[i].page_content.split('\n'))
    # Regular expression patterns for email and phone numbers
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    phone_pattern = r'\d{3}-\d{3}-\d{4}'
    # Finding all matches for email and phone numbers
    emails = re.findall(email_pattern, data)
    phones = re.findall(phone_pattern, data)
    # Splitting the data based on emails to capture the text segments that may contain phones
    split_data = re.split(email_pattern, data)

    email_phone_pairs = []
    phone_index = 0

    for i, segment in enumerate(split_data[:-1]):  # ignore the last segment as it comes after the last email
        email = emails[i]
        # Check if the next segment contains a phone number
        if phone_index < len(phones) and phones[phone_index] in segment:
            email_phone_pairs.append(f"{email} {phones[phone_index]}")
            phone_index += 1
        else:
            email_phone_pairs.append(f"{email} N/A")

    # Print the results
    for pair in email_phone_pairs:
       blank_document.page_content = pair
       new_professor_digital.append(copy.deepcopy(blank_document)) 

In [46]:
new_professor_digital

[Document(page_content='anayaj@wit.edu N/A'),
 Document(page_content='andersonl@wit.edu 617-989-4182'),
 Document(page_content='andersone10@wit.edu 617-989-5008'),
 Document(page_content='andersonk12@wit.edu 617-989-4296'),
 Document(page_content='andren@wit.edu N/A'),
 Document(page_content='andresn@wit.edu 617-989-4457'),
 Document(page_content='anwaruddinm@wit.edu N/A'),
 Document(page_content='ardolinoe@wit.edu N/A'),
 Document(page_content='arecyc@wit.edu 617-989-4808'),
 Document(page_content='arevalod@wit.edu 617-989-4537'),
 Document(page_content='armenisa@wit.edu 617-989-4003'),
 Document(page_content='armstrongm2@wit.edu 617-989-5341'),
 Document(page_content='aronovj@wit.edu N/A'),
 Document(page_content='askerneesee@wit.edu 617-989-4602'),
 Document(page_content='assab@wit.edu 617-989-4285'),
 Document(page_content='aureliov@wit.edu N/A'),
 Document(page_content='austinwilksa@wit.edu N/A'),
 Document(page_content='avelard@wit.edu N/A'),
 Document(page_content='averbecka@wit

In [47]:
print(len(new_professor_titles))
print(len(new_professor_digital))

696
696


In [48]:
professors = []
for i in range(len(new_professor_titles)):
    temp_prof = copy.deepcopy(new_professor_titles[i])
    temp_prof.page_content = temp_prof.page_content + ' ' + new_professor_digital[i].page_content
    professors.append(temp_prof)

In [49]:
professors

[Document(page_content='Julito Anaya Maintenance A Worker, Physical Plant anayaj@wit.edu N/A'),
 Document(page_content='Lenny Anderson Associate Professor, Civil Engineering, School of Engineering andersonl@wit.edu 617-989-4182'),
 Document(page_content='Ean Anderson Student Success Advisor, Success Studio andersone10@wit.edu 617-989-5008'),
 Document(page_content='Kayla Anderson Student Success Advisor, Success Studio andersonk12@wit.edu 617-989-4296'),
 Document(page_content='Nedret Andre Adjunct Instructor, Architecture, School of Architecture and Design andren@wit.edu N/A'),
 Document(page_content='Nicholas Andres Academic Coordinator, School of Architecture and Design andresn@wit.edu 617-989-4457'),
 Document(page_content='Mohammed Anwaruddin Adjunct Instructor, Computer Science, School of Computing and Data Science anwaruddinm@wit.edu N/A'),
 Document(page_content='Ed Ardolino Coach, Athletics ardolinoe@wit.edu N/A'),
 Document(page_content='Christina Arecy Assistant Director-Div

In [61]:
# send all faculty/professor info to file path
with open('Data/professors.txt', 'w') as file:
    for line in professors:
        file.write(line.page_content + '\n')

In [50]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(professors, embeddings, index_name=PINECONE_INDEX_NAME)

In [51]:
query = "Micah"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

Micah Schuster Associate Professor, Computer Science, School of Computing and Data Science schusterm@wit.edu 617-989-4297


In [52]:
AI_TWENTYONE_API_KEY = "XaZ0TkUk0Z8NiUMZGxcaD1HPmdeV5V1C"

In [53]:
os.environ["AI21_API_KEY"] = AI_TWENTYONE_API_KEY

In [54]:
from langchain_pinecone import Pinecone
from langchain_ai21 import AI21ContextualAnswers

professorSearch = Pinecone(
    index_name=PINECONE_INDEX_NAME,
    embedding=OpenAIEmbeddings()
)

In [87]:
tsm = AI21ContextualAnswers()
chain = tsm | StrOutputParser()

question = "What is Frank Kreimendahl's email adress?"
query_result = professorSearch.max_marginal_relevance_search(question, k=5, fetch_k=10)
found_docs = ""


In [88]:
query_result

[Document(page_content='Frank Kreimendahl Assistant Professor, Applied Mathematics, School of Computing and Data Science kreimendahlf@wit.edu 617-989-5027'),
 Document(page_content='Kelly McCarthy Coach mccarthyk15@wit.edu N/A'),
 Document(page_content='Laura Schmidt Adjunct Instructor, Architecture, School of Architecture and Design schmidtl1@wit.edu N/A'),
 Document(page_content='Leart Krasniqi Adjunct Instructor, Computer Science, School of Computing and Data Science krasniqil@wit.edu N/A'),
 Document(page_content='Koorosh Firouzbakht Two Semester Appointment, Computer Science, School of Computing and Data Science firouzbakhtk@wit.edu 617-989-4397')]

In [89]:
for i, doc in enumerate(query_result):
    found_docs = found_docs + doc.page_content + " "
response = chain.invoke(
    {"context": found_docs, "question": question},
)

In [90]:
print(response)

Frank Kreimendahl's email adress is kreimendahlf@wit.edu.
